## GenerativeAI4DS-II
## Healthcare Lab. Question and Answering services


### In this lab you will learn how to implement RAG architectures to process healthcare information and implement Q&A services over medical records

### Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)

In [2]:
#OPENAI_API_KEY="YOU-NEED-YOUR-OWN-API"

In [3]:
!pip install --upgrade langchain;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.6 MB/s eta 0:00:00


In [4]:

!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [5]:
!pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 757.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.4 MB/s eta 0:00:00


In [6]:
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.7 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=2ea06db1e2648c62ca2770d6298dc4e94fc2f4284ebba925cbec1a15

In [7]:
!pip install --upgrade --quiet  "docarray"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 3.8 MB/s eta 0:00:00


In [8]:
!pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.6 MB/s eta 0:00:00


In [9]:
import os
import openai



In [10]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_community.document_loaders import DataFrameLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.llms import OpenAI
import pandas as pd

### 1. Let's load a small subset of medical records

In [35]:
df=pd.read_csv("https://github.com/thousandoaks/Python4DS-I/raw/main/datasets/HealthcareDataset_PublicRelease.csv",sep=',',parse_dates=['StartDate','EndDate','BirthDate'])

In [36]:
pd.options.display.max_colwidth = 300
df.head(20)

,Id,MemberName,MemberID,County,MedicalClaim,ClaimItem,HospitalName,HospitalType,StartDate,EndDate,PrincipalDiagnosisDesc,PrincipalDiagnosis,RevenueCodeDesc,RevenueCode,TypeFlag,BirthDate,TotalExpenses
0,634363,e659f3f4,6a380a28,6f943458,c1e3436737c77899,18,04b77561,HOSPITAL,2020-01-08,2020-01-08,Epigastric pain,R10.13,DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING,636.0,ER,1967-05-13,15.148
1,634364,e659f3f4,6a380a28,6f943458,c1e3436737c77899,21,04b77561,HOSPITAL,2020-01-08,2020-01-08,Epigastric pain,R10.13,DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING,636.0,ER,1967-05-13,3.073
2,634387,e659f3f4,6a380a28,6f943458,c1e3436737c77899,10,04b77561,HOSPITAL,2020-01-08,2020-01-08,Epigastric pain,R10.13,LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY,305.0,ER,1967-05-13,123.900
3,634388,e659f3f4,6a380a28,6f943458,c1e3436737c77899,20,04b77561,HOSPITAL,2020-01-08,2020-01-08,Epigastric pain,R10.13,DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING,636.0,ER,1967-05-13,7.511
4,634389,e659f3f4,6a380a28,6f943458,c1e3436737c77899,19,04b77561,HOSPITAL,2020-01-08,2020-01-08,Epigastric pain,R10.13,DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING,636.0,ER,1967-05-13,8.631
5,634390,e659f3f4,6a380a28,6f943458,c1e3436737c77899,2,04b77561,HOSPITAL,2020-01-08,2020-01-08,Epigastric pain,R10.13,LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY,301.0,ER,1967-05-13,263.200
6,634391,e659f3f4,6a380a28,6f943458,c1e3436737c77899,6,04b77561,HOSPITAL,2020-01-08,2020-01-08,Epigastric pain,R10.13,LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY,301.0,ER,1967-05-13,44.100
7,634392,e659f3f4,6a380a28,6f943458,c1e3436737c77899,12,04b77561,HOSPITAL,2020-01-08,2020-01-08,Epigastric pain,R10.13,RADIOLOGY - DIAGNOSTIC: CHEST X-RAY,324.0,ER,1967-05-13,364.000
8,634393,e659f3f4,6a380a28,6f943458,c1e3436737c77899,15,04b77561,HOSPITAL,2020-01-08,2020-01-08,Epigastric pain,R10.13,EMERGENCY ROOM,450.0,ER,1967-05-13,789.600
9,634394,e659f3f4,6a380a28,6f943458,c1e3436737c77899,17,04b77561,HOSPITAL,2020-01-08,2020-01-08,Epigastric pain,R10.13,EMERGENCY ROOM,450.0,ER,1967-05-13,478.100


In [111]:
MedicalRecords=df.sort_values(by='MemberName').head(200).groupby(["MedicalClaim","MemberName","PrincipalDiagnosisDesc","PrincipalDiagnosis"]).agg({"TotalExpenses": ["sum"],"RevenueCodeDesc":' '.join}).reset_index()





In [112]:
MedicalRecords.columns

MultiIndex([(          'MedicalClaim',     ''),
            (            'MemberName',     ''),
            ('PrincipalDiagnosisDesc',     ''),
            (    'PrincipalDiagnosis',     ''),
            (         'TotalExpenses',  'sum'),
            (       'RevenueCodeDesc', 'join')],
           )

In [113]:
# We flatten the multindex into a regular DataFrame
MedicalRecords.columns=MedicalRecords.columns.get_level_values(0)

In [114]:
MedicalRecords.head(3)

,MedicalClaim,MemberName,PrincipalDiagnosisDesc,PrincipalDiagnosis,TotalExpenses,RevenueCodeDesc
0,01db6f5467574d6e,00bdc46e,Hypertensive heart diseas,I11.0,111436.402,LABORATORY - CLINICAL DIAGNOSTIC: UROLOGY MEDICAL/SURGICAL/GYN PULMONARY FUNCTION RADIOLOGY - DIAGNOSTIC: CHEST X-RAY PHYSICAL THERAPY: EVALUATION/RE-EVALUATION PHYSICAL THERAPY RESPIRATORY SERVICES EMERGENCY ROOM CT SCAN: HEAD LABORATORY - CLINICAL DIAGNOSTIC: BACTERIOLOGY/MICROBIOLOGY CT SCAN ...
1,11c6f93bdbb127d8,00f62513,Rectal abscess,K61.1,46470.151,EKG/ECG BLOOD STORAGE/PROCESSING ANESTHESIA OPERATING ROOM SERVICES CT SCAN
2,3076d98ae812aec7,00f62513,Abnormal finding of blood,R79.9,4739.028,EMERGENCY ROOM LABORATORY - CLINICAL DIAGNOSTIC LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY MEDICAL/SURGICAL SUPPLIES: STERILE SUPPLIES LABORATORY - CLINICAL DI...


In [146]:
# prompt: Using dataframe MedicalRecords: count number of different MemberName

MedicalRecords.groupby('MemberName').count()

,MedicalClaim,PrincipalDiagnosisDesc,PrincipalDiagnosis,TotalExpenses,RevenueCodeDesc,MedicalInformation
MemberName,,,,,,
000e9b80,1,1,1,1,1,1
002451eb,1,1,1,1,1,1
003e47b4,1,1,1,1,1,1
00a90ae2,1,1,1,1,1,1
00bdc46e,3,3,3,3,3,3
00c10ea4,3,3,3,3,3,3
00f62513,3,3,3,3,3,3


In [115]:
MedicalRecords=MedicalRecords.astype(str)

In [120]:
MedicalRecords['MedicalInformation']='MedicalClaim: '+MedicalRecords['MedicalClaim']+ ' # ' +'MemberName: ' +MedicalRecords['MemberName']+ ' # ' +'PrincipalDiagnosisDesc: ' +MedicalRecords['PrincipalDiagnosisDesc']+ ' # ' +'PrincipalDiagnosis: ' +MedicalRecords['PrincipalDiagnosis']+ ' # ' +'TotalExpenses: ' +MedicalRecords['TotalExpenses']+ ' # ' +'RevenueCodeDesc: ' +MedicalRecords['RevenueCodeDesc']



In [121]:
MedicalRecords.head(2)

,MedicalClaim,MemberName,PrincipalDiagnosisDesc,PrincipalDiagnosis,TotalExpenses,RevenueCodeDesc,MedicalInformation
0,01db6f5467574d6e,00bdc46e,Hypertensive heart diseas,I11.0,111436.402,LABORATORY - CLINICAL DIAGNOSTIC: UROLOGY MEDICAL/SURGICAL/GYN PULMONARY FUNCTION RADIOLOGY - DIAGNOSTIC: CHEST X-RAY PHYSICAL THERAPY: EVALUATION/RE-EVALUATION PHYSICAL THERAPY RESPIRATORY SERVICES EMERGENCY ROOM CT SCAN: HEAD LABORATORY - CLINICAL DIAGNOSTIC: BACTERIOLOGY/MICROBIOLOGY CT SCAN ...,MedicalClaim: 01db6f5467574d6e # MemberName: 00bdc46e # PrincipalDiagnosisDesc: Hypertensive heart diseas # PrincipalDiagnosis: I11.0 # TotalExpenses: 111436.402 # RevenueCodeDesc: LABORATORY - CLINICAL DIAGNOSTIC: UROLOGY MEDICAL/SURGICAL/GYN PULMONARY FUNCTION RADIOLOGY - DIAGNOSTIC: CHEST X-R...
1,11c6f93bdbb127d8,00f62513,Rectal abscess,K61.1,46470.151000000005,EKG/ECG BLOOD STORAGE/PROCESSING ANESTHESIA OPERATING ROOM SERVICES CT SCAN,MedicalClaim: 11c6f93bdbb127d8 # MemberName: 00f62513 # PrincipalDiagnosisDesc: Rectal abscess # PrincipalDiagnosis: K61.1 # TotalExpenses: 46470.151000000005 # RevenueCodeDesc: EKG/ECG BLOOD STORAGE/PROCESSING ANESTHESIA OPERATING ROOM SERVICES CT SCAN


In [122]:
loader = DataFrameLoader(MedicalRecords, page_content_column='MedicalInformation')

In [123]:
MedicalRecordsCatalog=loader.load()

In [124]:
len(MedicalRecordsCatalog)

13

In [125]:
MedicalRecordsCatalog[0]

Document(metadata={'MedicalClaim': '01db6f5467574d6e', 'MemberName': '00bdc46e', 'PrincipalDiagnosisDesc': 'Hypertensive heart diseas', 'PrincipalDiagnosis': 'I11.0', 'TotalExpenses': '111436.402', 'RevenueCodeDesc': 'LABORATORY - CLINICAL DIAGNOSTIC: UROLOGY MEDICAL/SURGICAL/GYN PULMONARY FUNCTION RADIOLOGY - DIAGNOSTIC: CHEST X-RAY PHYSICAL THERAPY: EVALUATION/RE-EVALUATION PHYSICAL THERAPY RESPIRATORY SERVICES EMERGENCY ROOM CT SCAN: HEAD LABORATORY - CLINICAL DIAGNOSTIC: BACTERIOLOGY/MICROBIOLOGY CT SCAN LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY PHARMACY MEDICAL/SURGICAL SUPPLIES: STERILE SUPPLIES LABORATORY - CLINICAL DIAGNOSTIC RADIOLOGY - DIAGNOSTIC PHYSICAL THERAPY: VISIT CHARGE LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY INTERMEDIATE ICU DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY EKG/ECG MEDICAL/SURGICAL SUPPLIES OTHER IMAGING SERVICES: ULTRASOUND'}, page_content='MedicalClaim: 01db6f5467574d6e # MemberName: 00bdc46

In [126]:
MedicalRecordsCatalog[1]

Document(metadata={'MedicalClaim': '11c6f93bdbb127d8', 'MemberName': '00f62513', 'PrincipalDiagnosisDesc': 'Rectal abscess', 'PrincipalDiagnosis': 'K61.1', 'TotalExpenses': '46470.151000000005', 'RevenueCodeDesc': 'EKG/ECG BLOOD STORAGE/PROCESSING ANESTHESIA OPERATING ROOM SERVICES CT SCAN'}, page_content='MedicalClaim: 11c6f93bdbb127d8 # MemberName: 00f62513 # PrincipalDiagnosisDesc: Rectal abscess # PrincipalDiagnosis: K61.1 # TotalExpenses: 46470.151000000005 # RevenueCodeDesc: EKG/ECG BLOOD STORAGE/PROCESSING ANESTHESIA OPERATING ROOM SERVICES CT SCAN')

### 2. Let's split these documents

In [127]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, add_start_index=True
)

In [128]:
MedicalRecordsCatalogItems = text_splitter.split_documents(MedicalRecordsCatalog)

In [129]:
len(MedicalRecordsCatalogItems)

14

In [130]:
MedicalRecordsCatalogItems[0]

Document(metadata={'MedicalClaim': '01db6f5467574d6e', 'MemberName': '00bdc46e', 'PrincipalDiagnosisDesc': 'Hypertensive heart diseas', 'PrincipalDiagnosis': 'I11.0', 'TotalExpenses': '111436.402', 'RevenueCodeDesc': 'LABORATORY - CLINICAL DIAGNOSTIC: UROLOGY MEDICAL/SURGICAL/GYN PULMONARY FUNCTION RADIOLOGY - DIAGNOSTIC: CHEST X-RAY PHYSICAL THERAPY: EVALUATION/RE-EVALUATION PHYSICAL THERAPY RESPIRATORY SERVICES EMERGENCY ROOM CT SCAN: HEAD LABORATORY - CLINICAL DIAGNOSTIC: BACTERIOLOGY/MICROBIOLOGY CT SCAN LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY PHARMACY MEDICAL/SURGICAL SUPPLIES: STERILE SUPPLIES LABORATORY - CLINICAL DIAGNOSTIC RADIOLOGY - DIAGNOSTIC PHYSICAL THERAPY: VISIT CHARGE LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY INTERMEDIATE ICU DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY EKG/ECG MEDICAL/SURGICAL SUPPLIES OTHER IMAGING SERVICES: ULTRASOUND', 'start_index': 0}, page_content='MedicalClaim: 01db6f5467574d6e # M

In [131]:
MedicalRecordsCatalogItems[1]

Document(metadata={'MedicalClaim': '11c6f93bdbb127d8', 'MemberName': '00f62513', 'PrincipalDiagnosisDesc': 'Rectal abscess', 'PrincipalDiagnosis': 'K61.1', 'TotalExpenses': '46470.151000000005', 'RevenueCodeDesc': 'EKG/ECG BLOOD STORAGE/PROCESSING ANESTHESIA OPERATING ROOM SERVICES CT SCAN', 'start_index': 0}, page_content='MedicalClaim: 11c6f93bdbb127d8 # MemberName: 00f62513 # PrincipalDiagnosisDesc: Rectal abscess # PrincipalDiagnosis: K61.1 # TotalExpenses: 46470.151000000005 # RevenueCodeDesc: EKG/ECG BLOOD STORAGE/PROCESSING ANESTHESIA OPERATING ROOM SERVICES CT SCAN')

In [90]:
MedicalRecordsCatalogItems[2]

Document(metadata={'MedicalClaim': '3076d98ae812aec7', 'MemberName': '00f62513', 'PrincipalDiagnosisDesc': 'Abnormal finding of blood', 'PrincipalDiagnosis': 'R79.9', 'TotalExpenses': 4739.028, 'start_index': 0}, page_content='EMERGENCY ROOM LABORATORY - CLINICAL DIAGNOSTIC LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY MEDICAL/SURGICAL SUPPLIES: STERILE SUPPLIES LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY')

### 2. Let's insert the catalog into a Vector Store

In [132]:
from langchain.embeddings import OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [133]:
from langchain_community.vectorstores import DocArrayInMemorySearch
db = DocArrayInMemorySearch.from_documents(MedicalRecordsCatalogItems, embeddings_model)

### 3. Let's query the database

In [163]:
query = "EMERGENCY"

In [175]:
docs = db.similarity_search(query,k=10)

In [176]:
len(docs)

10

In [177]:
docs[0]

Document(metadata={'MedicalClaim': '6016589b30e19fd0', 'MemberName': '00c10ea4', 'PrincipalDiagnosisDesc': 'Myalgia unspecified site', 'PrincipalDiagnosis': 'M79.10', 'TotalExpenses': '6728.981', 'RevenueCodeDesc': 'LABORATORY - CLINICAL DIAGNOSTIC DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING EMERGENCY ROOM EKG/ECG MEDICAL/SURGICAL SUPPLIES: STERILE SUPPLIES LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY EMERGENCY ROOM EMERGENCY ROOM PHARMACY', 'start_index': 0}, page_content='MedicalClaim: 6016589b30e19fd0 # MemberName: 00c10ea4 # PrincipalDiagnosisDesc: Myalgia unspecified site # PrincipalDiagnosis: M79.10 # TotalExpenses: 6728.981 # RevenueCodeDesc: LABORATORY - CLINICAL DIAGNOSTIC DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING EMERGENCY ROOM EKG/ECG MEDICAL/SURGICAL SUPPLIES: STERILE SUPPLIES LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY EMERGENCY ROOM EMERGENCY ROOM PHARMACY'

In [178]:
docs[1]

Document(metadata={'MedicalClaim': '3076d98ae812aec7', 'MemberName': '00f62513', 'PrincipalDiagnosisDesc': 'Abnormal finding of blood', 'PrincipalDiagnosis': 'R79.9', 'TotalExpenses': '4739.028', 'RevenueCodeDesc': 'EMERGENCY ROOM LABORATORY - CLINICAL DIAGNOSTIC LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY MEDICAL/SURGICAL SUPPLIES: STERILE SUPPLIES LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY', 'start_index': 0}, page_content='MedicalClaim: 3076d98ae812aec7 # MemberName: 00f62513 # PrincipalDiagnosisDesc: Abnormal finding of blood # PrincipalDiagnosis: R79.9 # TotalExpenses: 4739.028 # RevenueCodeDesc: EMERGENCY ROOM LABORATORY - CLINICAL DIAGNOSTIC LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY MEDICAL/SURGICAL SUPPLIES:

In [179]:
docs[2]

Document(metadata={'MedicalClaim': 'd0b551e232e1b6f1', 'MemberName': '00bdc46e', 'PrincipalDiagnosisDesc': 'Hypertensive heart diseas', 'PrincipalDiagnosis': 'I11.0', 'TotalExpenses': '36030.337', 'RevenueCodeDesc': 'PHYSICAL THERAPY: EVALUATION/RE-EVALUATION MEDICAL/SURGICAL SUPPLIES MEDICAL/SURGICAL/GYN LABORATORY - CLINICAL DIAGNOSTIC PHYSICAL THERAPY LABORATORY - CLINICAL DIAGNOSTIC: BACTERIOLOGY/MICROBIOLOGY LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY EMERGENCY ROOM LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY OTHER DIAGNOSTIC SERVICES: PERIPHERAL VASCULAR LAB LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY RADIOLOGY - DIAGNOSTIC: CHEST X-RAY LABORATORY - CLINICAL DIAGNOSTIC: UROLOGY DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING PHARMACY MEDICAL/SURGICAL SUPPLIES: STERILE SUPPLIES EKG/ECG', 'start_index': 0}, page_content='MedicalClaim: d0b551e232e1b6f1 # MemberName: 00bdc46e # PrincipalDiagnosisDesc: Hypertensive heart diseas # PrincipalDiagnosis: I11.0 # TotalExpenses: 3

In [180]:
docs[3]

Document(metadata={'MedicalClaim': 'f92bb137b39f997b', 'MemberName': '00f62513', 'PrincipalDiagnosisDesc': 'Sepsis unspecified organi', 'PrincipalDiagnosis': 'A41.9', 'TotalExpenses': '115949.302', 'RevenueCodeDesc': 'OTHER DIAGNOSTIC SERVICES: PERIPHERAL VASCULAR LAB EKG/ECG RECOVERY ROOM DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING PULMONARY FUNCTION PHARMACY: IV SOLUTIONS PHYSICAL THERAPY BLOOD STORAGE/PROCESSING: BLOOD ADMINISTRATION (EG. TRANSFUSION) ANESTHESIA CT SCAN RADIOLOGY - DIAGNOSTIC: CHEST X-RAY LABORATORY - CLINICAL DIAGNOSTIC: BACTERIOLOGY/MICROBIOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY BLOOD STORAGE/PROCESSING OPERATING ROOM SERVICES', 'start_index': 0}, page_content='MedicalClaim: f92bb137b39f997b # MemberName: 00f62513 # PrincipalDiagnosisDesc: Sepsis unspecified organi # PrincipalDiagnosis: A41.9 # TotalExpenses: 115949.302 # RevenueCodeDesc: OTHER DIAGNOSTIC SERVICES: PERIPHERAL VASCULAR LAB EKG/ECG RECOVERY ROOM DRUGS REQUIRE SPECIFIC ID: DRU

In [181]:
docs[4]

Document(metadata={'MedicalClaim': '3d3f3e6226e870f9', 'MemberName': '00c10ea4', 'PrincipalDiagnosisDesc': 'Occlusion and stenosis of', 'PrincipalDiagnosis': 'I65.21', 'TotalExpenses': '74796.841', 'RevenueCodeDesc': 'DRUGS REQUIRE SPECIFIC ID: SELF ADMIN DRUGS (INSULIN ADMIN IN EMERGENCY-DIABETES COMA) PHYSICAL THERAPY: EVALUATION/RE-EVALUATION LABORATORY - PATHOLOGY ANESTHESIA DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING OPERATING ROOM SERVICES LABORATORY - CLINICAL DIAGNOSTIC LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY LABORATORY - CLINICAL DIAGNOSTIC: BACTERIOLOGY/MICROBIOLOGY RECOVERY ROOM INTERMEDIATE ICU MEDICAL/SURGICAL SUPPLIES: OTHER IMPLANTS PHARMACY LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY MEDICAL/SURGICAL SUPPLIES: STERILE SUPPLIES PHARMACY: OTHER MEDICAL/SURGICAL SUPPLIES LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY INPATIENT DEMODIALYSIS', 'start_index': 0}, page_content='MedicalClaim: 3d3f3e6226e870f9 # MemberName: 00c10ea4 # PrincipalDiagnosisDesc: Occl

In [182]:
docs[5]

Document(metadata={'MedicalClaim': 'fe3763cb500a3328', 'MemberName': '00c10ea4', 'PrincipalDiagnosisDesc': 'Chest pain unspecified', 'PrincipalDiagnosis': 'R07.9', 'TotalExpenses': '21864.479', 'RevenueCodeDesc': 'EKG/ECG LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY PHARMACY OTHER DIAGNOSTIC SERVICES: PERIPHERAL VASCULAR LAB INPATIENT DEMODIALYSIS MEDICAL/SURGICAL/GYN MEDICAL/SURGICAL SUPPLIES: STERILE SUPPLIES DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING LABORATORY - CLINICAL DIAGNOSTIC: BACTERIOLOGY/MICROBIOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY TREATMENT/OBSERVATION ROOM: OBSERVATION ROOM CLINIC LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY RADIOLOGY - DIAGNOSTIC: CHEST X-RAY EMERGENCY ROOM PHARMACY: OTHER LABORATORY - CLINICAL DIAGNOSTIC', 'start_index': 0}, page_content='MedicalClaim: fe3763cb500a3328 # MemberName: 00c10ea4 # PrincipalDiagnosisDesc: Chest pain unspecified # PrincipalDiagnosis: R07.9 # TotalExpenses: 21864.479 # RevenueCodeDesc: EKG/ECG LABORAT

In [183]:
docs[6]

Document(metadata={'MedicalClaim': '01db6f5467574d6e', 'MemberName': '00bdc46e', 'PrincipalDiagnosisDesc': 'Hypertensive heart diseas', 'PrincipalDiagnosis': 'I11.0', 'TotalExpenses': '111436.402', 'RevenueCodeDesc': 'LABORATORY - CLINICAL DIAGNOSTIC: UROLOGY MEDICAL/SURGICAL/GYN PULMONARY FUNCTION RADIOLOGY - DIAGNOSTIC: CHEST X-RAY PHYSICAL THERAPY: EVALUATION/RE-EVALUATION PHYSICAL THERAPY RESPIRATORY SERVICES EMERGENCY ROOM CT SCAN: HEAD LABORATORY - CLINICAL DIAGNOSTIC: BACTERIOLOGY/MICROBIOLOGY CT SCAN LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY PHARMACY MEDICAL/SURGICAL SUPPLIES: STERILE SUPPLIES LABORATORY - CLINICAL DIAGNOSTIC RADIOLOGY - DIAGNOSTIC PHYSICAL THERAPY: VISIT CHARGE LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY INTERMEDIATE ICU DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY EKG/ECG MEDICAL/SURGICAL SUPPLIES OTHER IMAGING SERVICES: ULTRASOUND', 'start_index': 0}, page_content='MedicalClaim: 01db6f5467574d6e # M

In [184]:
docs[7]

Document(metadata={'MedicalClaim': 'bd40b943533deb7b', 'MemberName': '002451eb', 'PrincipalDiagnosisDesc': 'Chronic atrial fibrillati', 'PrincipalDiagnosis': 'I48.20', 'TotalExpenses': '23153.564000000002', 'RevenueCodeDesc': 'LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY PREVENTIVE CARE SERVICES: ADMIN. OF VACCINE LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY EKG/ECG TREATMENT/OBSERVATION ROOM: OBSERVATION ROOM RADIOLOGY - DIAGNOSTIC: CHEST X-RAY EKG/ECG LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY EMERGENCY ROOM LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY LABORATORY - CLINICAL DIAGNOSTIC LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY CARDIOLOGY: ECHOCARDIOLOGY DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING OTHER THERAPEUTIC SERV LABORATORY - CLINIC

In [185]:
docs[8]

Document(metadata={'MedicalClaim': '3da6d9035cb53f0d', 'MemberName': '00a90ae2', 'PrincipalDiagnosisDesc': 'Open bite of right hand i', 'PrincipalDiagnosis': 'S61.451A', 'TotalExpenses': '9587.557', 'RevenueCodeDesc': 'LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY LABORATORY - CLINICAL DIAGNOSTIC: BACTERIOLOGY/MICROBIOLOGY DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY LABORATORY - CLINICAL DIAGNOSTIC PREVENTIVE CARE SERVICES: ADMIN. OF VACCINE IV THERAPY DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING EMERGENCY ROOM LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY', 'start_index': 0}, page_content='MedicalClaim: 3da6d9035cb53f0d # MemberName: 00a90ae2 # PrincipalDiagnosisDesc: Open bite of right hand i # PrincipalDiagnosis: S61.451A # TotalExpenses: 9587.557 # RevenueCodeDesc: LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY LABORATORY - CLINICAL DIAGNOSTIC: BACTERIOLOGY/MICROB

In [186]:
docs[9]

Document(metadata={'MedicalClaim': '8cc4cd548a13b44b', 'MemberName': '00bdc46e', 'PrincipalDiagnosisDesc': 'Incisional hernia with ob', 'PrincipalDiagnosis': 'K43.0', 'TotalExpenses': '35680.729', 'RevenueCodeDesc': 'MEDICAL/SURGICAL SUPPLIES MEDICAL/SURGICAL/GYN DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING LABORATORY - CLINICAL DIAGNOSTIC EMERGENCY ROOM LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY LABORATORY - CLINICAL DIAGNOSTIC: UROLOGY RADIOLOGY - DIAGNOSTIC CT SCAN PHARMACY PHYSICAL THERAPY: VISIT CHARGE EKG/ECG PHYSICAL THERAPY: EVALUATION/RE-EVALUATION PHYSICAL THERAPY', 'start_index': 0}, page_content='MedicalClaim: 8cc4cd548a13b44b # MemberName: 00bdc46e # PrincipalDiagnosisDesc: Incisional hernia with ob # PrincipalDiagnosis: K43.0 # TotalExpenses: 35680.729 # RevenueCodeDesc: MEDICAL/SURGICAL SUPPLIES MEDICAL/SURGICAL/GYN DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING LABORATORY - CLINICAL DIAGNOSTIC EMERGENCY

### 4. Let's implement a retriever

In [170]:
retriever = db.as_retriever()

In [171]:
qdocs = "\n\n".join([docs[i].page_content for i in range(len(docs))])


In [172]:
print(qdocs)

MedicalClaim: 6016589b30e19fd0 # MemberName: 00c10ea4 # PrincipalDiagnosisDesc: Myalgia unspecified site # PrincipalDiagnosis: M79.10 # TotalExpenses: 6728.981 # RevenueCodeDesc: LABORATORY - CLINICAL DIAGNOSTIC DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DETAIL CODING EMERGENCY ROOM EKG/ECG MEDICAL/SURGICAL SUPPLIES: STERILE SUPPLIES LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY EMERGENCY ROOM EMERGENCY ROOM PHARMACY

MedicalClaim: 3076d98ae812aec7 # MemberName: 00f62513 # PrincipalDiagnosisDesc: Abnormal finding of blood # PrincipalDiagnosis: R79.9 # TotalExpenses: 4739.028 # RevenueCodeDesc: EMERGENCY ROOM LABORATORY - CLINICAL DIAGNOSTIC LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY LABORATORY - CLINICAL DIAGNOSTIC: IMMUNOLOGY MEDICAL/SURGICAL SUPPLIES: STERILE SUPPLIES LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY

MedicalClaim: d0b551e232e1b6f1 # Member

In [173]:
llm = ChatOpenAI(temperature=0.0, model='gpt-4-0125-preview',openai_api_key=OPENAI_API_KEY)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## 5. Let's formulate a query

In [190]:
response = llm.call_as_llm(f"the following document lists Medical Events involving Emergency Room services. Your tasks are: \
(1) extract *ALL* events with medical code I11.0, (2) for each event extract the Membername and TotalExpenses, \
(3) for event provide a 1 paragraph summary of RevenueCodeDesc {qdocs}")

In [191]:
response

'For the medical event with code I11.0, the details are as follows:\n\n- **MemberName:** 00bdc46e\n- **TotalExpenses:** 36030.337\n\n**Summary of RevenueCodeDesc for MedicalClaim d0b551e232e1b6f1:**\n\nThis medical claim involves a patient diagnosed with Hypertensive heart disease (I11.0) and includes a wide range of services amounting to a total expense of $36,030.337. The services provided encompass various diagnostic and therapeutic procedures. Diagnostic services included a comprehensive set of laboratory clinical diagnostics such as bacteriology/microbiology, hematology, chemistry, immunology, urology, and a chest X-ray. Therapeutic services involved physical therapy evaluations and re-evaluations, emergency room services, and the provision of medical/surgical supplies including sterile supplies. Additionally, the claim covers drugs requiring specific identification and detail coding, highlighting the complexity and multifaceted nature of the treatment provided. This extensive arr

In [192]:
display(Markdown(response))

For the medical event with code I11.0, the details are as follows:

- **MemberName:** 00bdc46e
- **TotalExpenses:** 36030.337

**Summary of RevenueCodeDesc for MedicalClaim d0b551e232e1b6f1:**

This medical claim involves a patient diagnosed with Hypertensive heart disease (I11.0) and includes a wide range of services amounting to a total expense of $36,030.337. The services provided encompass various diagnostic and therapeutic procedures. Diagnostic services included a comprehensive set of laboratory clinical diagnostics such as bacteriology/microbiology, hematology, chemistry, immunology, urology, and a chest X-ray. Therapeutic services involved physical therapy evaluations and re-evaluations, emergency room services, and the provision of medical/surgical supplies including sterile supplies. Additionally, the claim covers drugs requiring specific identification and detail coding, highlighting the complexity and multifaceted nature of the treatment provided. This extensive array of services underscores the serious implications of hypertensive heart disease and the multifaceted approach required for its management in an emergency setting.

## 6. Let's formulate another query

In [193]:
response = llm.call_as_llm(f"the following document lists Medical Events involving Emergency Room services. Your tasks are: \
(1) compute total expenses {qdocs}")

In [194]:
display(Markdown(response))

To compute the total expenses from the provided medical claims, we will sum up the TotalExpenses from each claim.

1. MedicalClaim: 6016589b30e19fd0
   - TotalExpenses: $6,728.981

2. MedicalClaim: 3076d98ae812aec7
   - TotalExpenses: $4,739.028

3. MedicalClaim: d0b551e232e1b6f1
   - TotalExpenses: $36,030.337

4. MedicalClaim: f92bb137b39f997b
   - TotalExpenses: $115,949.302

5. MedicalClaim: 3d3f3e6226e870f9
   - TotalExpenses: $74,796.841

Adding these amounts together gives the total expenses:

Total = $6,728.981 + $4,739.028 + $36,030.337 + $115,949.302 + $74,796.841
Total = $238,244.489

Therefore, the total expenses for all the listed medical claims involving emergency room services is $238,244.489.